In [ ]:
import pandas as pd
import sqlite3

from airflow import DAG
from datetime import datetime
from airflow.utils.dates import days_ago
from airflow.operators.python import PythonOperator

#connect to DB
CON = sqlite3.connect('example.db')
# const
start = datetime(2021, 1, 1)
end = datetime(2021, 1, 4)

# extract currency from site
def extract_rate(date, **context) -> None:
  """ Extract currency rate from exchange"""
  base = 'EUR'
  symbols = 'USD'
  format = 'csv'
  url = f'https://api.exchangerate.host/timeseries?start_date={date}&end_date={date}&base={base}&symbols={symbols}&format={format}'
  return pd.read_csv(url)['rate'][0]
  

# extract data from GIT
def extract_data(date, data_file, **context) -> None:
  """date in string format yyyy-mm-dd"""
  url = f'https://raw.githubusercontent.com/dm-novikov/stepik_airflow_course/main/data_new/{date}.csv'
  
  pd.read_csv(url).to_csv(data_file)


# load to db
def load_data(conn, data_file, **context) -> None:
    """ Load to DB """
    data = pd.read_csv(data_file, usecols=['currency',	'value',	'date'])
    data.to_sql('cur_table', conn, if_exists='append', index=False)


# DAG
dag = DAG(dag_id='dag',
          default_args={'owner':'airflow'},
          schedule_interval='@daily',
          start_date=start,
          end_date=end)

extract_rate = PythonOperator(
    task_id='extract_rate',
    python_callable=extract_rate,
    dag=dag,
    do_xcom_push=True,
    op_kwargs={
      'date': '{{ ds }}'
    }
)

extract_data = PythonOperator(
    task_id='extract_data',
    python_callable=extract_data,
    dag=dag,
    op_kwargs={
      'date': '{{ ds }}',
      'data_file': '/tmp/data.csv'
    }
)

load_data = PythonOperator(
    task_id='load_data',
    python_callable=load_data,
    dag=dag,
    op_kwargs={
      'data_file': '/tmp/data.csv',
      'conn': CON
    }
)

# sequence
[extract_rate, extract_data] >> load_data